In [28]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
  llm=llm,
  max_token_limit=120,
  memory_key="chat_history"   # 템플릿안에 메모리가 history를 저장하도록 한 곳을 적어준다.
)

template="""

  You are helpful AI talking to a human

  {chat_history}
  Human:{question}
  You:
"""

chain = LLMChain(
  llm=llm,
  memory=memory,
  prompt=PromptTemplate.from_template(template),
  verbose=True  #chain  프롬프트 로그를 확인 할 수 있다
  return_messages=True  #문자열로 바꾸지 말고 실제 메세지로 바꿔달라는 것을 의미
)


chain.predict(question="My name is Heon")





> Entering new LLMChain chain...
Prompt after formatting:


  You are helpful AI talking to a human

  
  Human:My name is Heon
  You:


> Finished chain.


'Hello Heon! How can I assist you today?'

In [29]:
chain.predict(question="I live in Seoul")




> Entering new LLMChain chain...
Prompt after formatting:


  You are helpful AI talking to a human

  Human: My name is Heon
AI: Hello Heon! How can I assist you today?
  Human:I live in Seoul
  You:


> Finished chain.


"That's great to hear! How can I assist you with information or tasks related to Seoul?"

In [30]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:


  You are helpful AI talking to a human

  Human: My name is Heon
AI: Hello Heon! How can I assist you today?
Human: I live in Seoul
AI: That's great to hear! How can I assist you with information or tasks related to Seoul?
  Human:What is my name?
  You:


> Finished chain.


'Your name is Heon.'

In [27]:
memory.load_memory_variables({})

{'history': "System: Heon introduces themselves as living in Seoul. The AI responds by acknowledging Seoul as a vibrant city with a rich history and culture, asking Heon what they enjoy most about living there.\nHuman: What is my name?\nAI: I'm sorry, I do not have access to personal information such as your name."}

In [38]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
  llm=llm,
  max_token_limit=120,
  memory_key="chat_history" ,  # 템플릿안에 메모리가 history를 저장하도록 한 곳을 적어준다.
  return_messages=True  #문자열로 바꾸지 말고 실제 메세지로 바꿔달라는 것을 의미
)

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful AI talking to a human"),
  MessagesPlaceholder(variable_name="chat_history"), #누가 보냈는지 알 수 없는, 예측하기 어려운 메세지의 양과 제한 없는 양의 메세지를 가질 수 있다. 
  ("human"," {question}"),
])


chain = prompt|llm

chain.invoke({
  "chat_history":memory.load_memory_variables({})["chat_history"],
  "question":"My name is Heon"
})







AIMessage(content='Hello Heon! How can I assist you today?')

In [43]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
  llm=llm,
  max_token_limit=120,
  return_messages=True  #문자열로 바꾸지 말고 실제 메세지로 바꿔달라는 것을 의미
)

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful AI talking to a human"),
  MessagesPlaceholder(variable_name="history"), #누가 보냈는지 알 수 없는, 예측하기 어려운 메세지의 양과 제한 없는 양의 메세지를 가질 수 있다. 
  ("human"," {question}"),
])

def load_memory(_):
  return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory)| prompt|llm

def invoke_chain(question):
  result = chain.invoke(
    {"question":question}
  )
  memory.save_context({"input":question}, {"output":result.content})
  print(result)

invoke_chain("My name is Heon")
invoke_chain("What is my name?")

content='Hello Heon! How can I assist you today?'
content='Your name is Heon. How can I assist you today, Heon?'
